## <span style="color:#ff5f27">👨🏻‍🏫 Build Index </span>

In this notebook you will create a feature group for your candidate embeddings.

## <span style="color:#ff5f27">📝 Imports </span>

In [1]:
import tensorflow as tf
import pprint
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

2024-04-21 23:02:33.030999: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-21 23:02:33.032950: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-21 23:02:33.119546: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 23:02:33.119599: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 23:02:33.121415: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [2]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/398
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27">🎯 Compute Candidate Embeddings </span>

You start by computing candidate embeddings for all items in the training data.

First, you load your candidate model. Recall that you uploaded it to the Hopsworks Model Registry in the previous notebook. If you don't have the model locally you can download it from the Model Registry using the following code:

In [3]:
model = mr.get_model(
    name="candidate_model",
    version=1,
)
model_path = model.download()

In [4]:
candidate_model = tf.saved_model.load(model_path)

2024-04-21 23:02:42.601870: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:03:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-21 23:02:42.602114: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Next you compute the embeddings of all candidate videos that were used to train the retrieval model.

In [5]:
feature_view = fs.get_feature_view(
    name="retrieval",
    version=1,
)

In [6]:
train_df, val_df, test_df, _, _, _ = feature_view.train_validation_test_split(
    validation_size=0.1, 
    test_size=0.1,
    description='Retrieval dataset splits',
)
train_df.head(3)

Finished: Reading data from Hopsworks, using ArrowFlight (75.46s) 


,interaction_id,user_id,gender,age,country,video_id,category,views,likes,video_length
0,6970-12-7050,HQ655Q,Female,80,Senegal,1XR19H,News,59892,26369,175
1,7575-45-0675,CV657Y,Female,46,Denmark,7MI02X,Dance,82199,50628,107
2,7107-81-7490,ML423N,Female,30,Liberia,5YC51I,Music,331845,147385,171


In [7]:
# Get the list of input features for the candidate model from the model schema
model_schema = model.model_schema['input_schema']['columnar_schema']
candidate_features = [feat['name'] for feat in model_schema]

# Select the candidate features from the training DataFrame
item_df = train_df[candidate_features]

# Drop duplicate rows based on the 'article_id' column to get unique candidate items
item_df.drop_duplicates(subset="video_id", inplace=True)

item_df.head(3)

,video_id,category,views,likes,video_length
0,1XR19H,News,59892,26369,175
1,7MI02X,Dance,82199,50628,107
2,5YC51I,Music,331845,147385,171


In [8]:
# Create a TensorFlow dataset from the item DataFrame
item_ds = tf.data.Dataset.from_tensor_slices(
    {col: item_df[col] for col in item_df})

# Compute embeddings for all candidate items using the candidate_model
candidate_embeddings = item_ds.batch(2048).map(
    lambda x: (x["video_id"], candidate_model(x))
)

## <span style="color:#ff5f27">⚙️ Data Preparation </span>


In [9]:
# Concatenate all article IDs and embeddings from the candidate_embeddings dataset
all_article_ids = tf.concat([batch[0] for batch in candidate_embeddings], axis=0)
all_embeddings = tf.concat([batch[1] for batch in candidate_embeddings], axis=0)

# Convert tensors to numpy arrays
all_article_ids_np = all_article_ids.numpy()
all_embeddings_np = all_embeddings.numpy()

# Convert numpy arrays to lists
items_ids_list = all_article_ids_np.tolist()
embeddings_list = all_embeddings_np.tolist()

In [10]:
# Create a DataFrame
data_emb = pd.DataFrame({
    'video_id': items_ids_list, 
    'embeddings': embeddings_list,
})
data_emb['video_id'] = data_emb['video_id'].str.decode('utf-8')

data_emb.head()

,video_id,embeddings
0,1XR19H,"[-0.04627479240298271, 0.5420380234718323, -0...."
1,7MI02X,"[-0.04627479240298271, 0.5420380234718323, -0...."
2,5YC51I,"[-0.04627479240298271, 0.5420380234718323, -0...."
3,9RP17P,"[-0.04627479240298271, 0.5420380234718323, -0...."
4,4YQ61G,"[-0.04627479240298271, 0.5420380234718323, -0...."


## <span style="color:#ff5f27">🪄 Feature Group Creation </span>

Now you are ready to create a feature group for your candidate embeddings.

To begin with, you need to create your Embedding Index where you will specify the name of the embeddings feature and the embeddings length.
Then you attach this index to the FG.

In [11]:
from hsfs import embedding

# Create the Embedding Index
emb = embedding.EmbeddingIndex()

emb.add_embedding(
    "embeddings",                           # Embeddings feature name
    len(data_emb["embeddings"].iloc[0]),    # Embeddings length
)

In [12]:
# Get or create the 'candidate_embeddings_fg' feature group
candidate_embeddings_fg = fs.get_or_create_feature_group(
    name="candidate_embeddings_fg",
    embedding_index=emb,                    # Specify the Embedding Index
    primary_key=['video_id'],
    version=1,
    description='Embeddings for each video',
    online_enabled=True,
)

candidate_embeddings_fg.insert(data_emb)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/398/fs/335/fg/739717


Uploading Dataframe: 100.00% |█████████████████████████████████████████████████████████████████████████| Rows 24989/24989 | Elapsed Time: 00:09 | Remaining Time: 00:00


Launching job: candidate_embeddings_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/398/jobs/named/candidate_embeddings_fg_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f94479c95d0>, None)

## <span style="color:#ff5f27">🪄 Feature View Creation </span>


In [13]:
# Get or create the 'candidate_embeddings' feature view
feature_view = fs.get_or_create_feature_view(
    name="candidate_embeddings",
    version=1,
    description='Embeddings of each article',
    query=candidate_embeddings_fg.select(["video_id"]),
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/398/fs/335/fv/candidate_embeddings/version/1


---
## <span style="color:#ff5f27">⏩️ Next Steps </span>

At this point you have a recommender system that is able to generate a set of candidate videos for a user. However, many of these could be poor, as the candidate model was trained with only a few subset of the features. In the next notebook, you'll create a ranking dataset to train a *ranking model* to do more fine-grained predictions.